In [1]:
import findspark
findspark.init('drive/My Drive/spark-2.4.5-bin-hadoop2.7')

import pandas as pd

import pyspark
from pyspark.sql.types import *

from IPython.display import display

from collections import Counter
import json
import re

In [2]:
sparkConf = pyspark.SparkConf()
sparkConf.setAppName("WikiCSSH")
sparkConf.setMaster("local")
sparkConf.setAll([
    ("spark.local.dir", "./tmp",),
    ('spark.driver.memory', '10G')
])

sc = pyspark.SparkContext(conf=sparkConf)
sc.setCheckpointDir("./spark_checkpoints")
spark = pyspark.sql.SparkSession(sc)

In [3]:
df = spark.read.parquet("drive/My Drive/categorylinks_category_page.parquet")

In [4]:
df

DataFrame[cl_from: int, cl_to: string, cl_type: string, cat_id: int, cat_title: string, cat_pages: int, cat_subcats: int, cat_files: int, cat_page_id: int, cat_page_namespace: int, cat_page_title: string, cat_page_is_redirect: int, page_id: int, page_namespace: int, page_title: string, page_is_redirect: int]

In [6]:
dd=pd.read_csv('drive/My Drive/categories.csv')

In [7]:
dd = list(dd.itertuples(index=False, name=None))

In [8]:
df_categories = sc.parallelize(dd).toDF(StructType([
    StructField("cat_title", StringType(), False),
    StructField("min_level", IntegerType(), False)
]))

In [9]:
df_categories

DataFrame[cat_title: string, min_level: int]

In [10]:
df_cat_pages = df_categories.join(
    df.filter("cl_type=='page' and page_namespace == 0"),
    on="cat_title").toPandas().sort_values("min_level")

In [11]:
df_cat_pages.head(5)

,cat_title,min_level,cl_from,cl_to,cl_type,cat_id,cat_pages,cat_subcats,cat_files,cat_page_id,cat_page_namespace,cat_page_title,cat_page_is_redirect,page_id,page_namespace,page_title,page_is_redirect
749335,Information_science,1,2843041,Information_science,page,166907,247,26,0,10246341,100,Information_science,1,2843041,0,Information_professional,0
23246,Computer_science,1,44409131,Computer_science,page,98333,50,17,0,1472580,100,Computer_science,0,44409131,0,Technology_transfer_in_computer_science,0
23245,Computer_science,1,44409131,Computer_science,page,98333,50,17,0,691117,14,Computer_science,0,44409131,0,Technology_transfer_in_computer_science,0
23244,Computer_science,1,44409131,Computer_science,page,98333,50,17,0,7290,1,Computer_science,0,44409131,0,Technology_transfer_in_computer_science,0
23243,Computer_science,1,44409131,Computer_science,page,98333,50,17,0,5323,0,Computer_science,0,44409131,0,Technology_transfer_in_computer_science,0


In [14]:
df_redirects = spark.read.parquet("drive/My Drive/redirects_page.parquet").withColumnRenamed(
    "page_title", "redirect_page_title").drop("page_id")
df_redirects.head()

Row(redirect_id=1190, redirected_page_title='Alternate_history', redirect_page_title='Alternative_history')

In [15]:
df_cat_pages_spark = df_categories.join(
    df.filter("cl_type=='page' and page_namespace == 0"),
    on="cat_title")

re_page = df_redirects.join(
    df_cat_pages_spark.filter("page_namespace == 0").select(
        "page_title", "page_id"
    ),
    df_redirects.redirected_page_title == df_cat_pages_spark.page_title
).select("redirect_id", "redirect_page_title", "page_title", "page_id").distinct().toPandas()

In [16]:
len(re_page['redirect_page_title'].unique())

580313

In [17]:
len(set(re_page['redirect_page_title'].tolist()) | set(df_cat_pages['page_title'].tolist()))

752246

In [18]:
total = pd.merge(df_cat_pages[['cat_title','page_title', 'min_level']].drop_duplicates(), re_page, 'left')[['page_title', 'min_level', 
                                                                                                    'redirect_page_title']].drop_duplicates()